# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [39]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [40]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [44]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

Adding the "relative link" information is inportant to get the link of type "/team" "/api". If not it won't get these.

In [45]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [46]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [47]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/stepvideo-t2v',
 '/stepfun-ai/Step-Audio-Chat',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/Steveeeeeeen/Zonos',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/SakanaAI/AI-CUDA-Engineer-Archive',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 

In [36]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}]}

In [48]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'api page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}]}

In [43]:
website = Website("https://anthropic.com")
website.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/claude-3-7-sonnet',
 '/claude/sonnet',
 '/news/visible-extended-thinking',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/solutions/coding',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosu

In [37]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [61]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [62]:
print(get_all_details("https://edwarddonner.com"))

Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has hap

In [63]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [64]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [65]:
get_brochure_user_prompt("EdwardDonner", "https://edwarddonner.com")

'You are looking at a company called: EdwardDonner\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m pr

In [66]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [67]:
create_brochure("EdwardDonner", "https://edwarddonner.com")

# Edward Donner Company Brochure

## Welcome to Edward Donner

At Edward Donner, we are at the forefront of innovating how talent is sourced, understood, engaged, and managed using the power of AI. Co-founded by Ed, an enthusiastic coder and tech innovator with a passion for experimentation, we are committed to aligning individual talents with potential roles to enhance human prosperity.

### About Us

**Mission:** Our mission at Edward Donner is to transform the hiring landscape by helping people discover their true potential and pursue their reason for being. With a unique approach powered by Generative AI, we aim to match candidates with roles that inspire fulfillment and success, drawing inspiration from the concept of *Ikigai*. 

**Products:** Our patented matching model leverages groundbreaking, proprietary language models verticalized for talent sourcing, resulting in a faster and more accurate hiring process—no keywords required. We pride ourselves in having developed an award-winning platform that has gained significant recognition and a loyal customer base.

### Our Culture

At Edward Donner, we celebrate innovation, creativity, and collaboration. Our founder believes in fostering a workplace where technology and human connection meet. Ed's dedication to tackling real-world problems gives our team a sense of purpose and direction. Whether it’s collaborating on groundbreaking projects or enjoying a spirited game of Connect Four in the office, we maintain an environment that promotes growth, learning, and a bit of fun.

### Customers

Our customers are talent acquisition leaders and recruitment firms who leverage our tools to simplify and enhance their hiring processes. By focusing on delivering unmatched technological solutions and understanding their unique needs, we help organizations find the right people efficiently and effectively.

### Careers at Edward Donner

We are always looking for passionate individuals to join our team. If you are excited about working at the intersection of AI and human potential, we invite you to connect with us. We value diverse skill sets, especially in software engineering, AI, and data science, and provide opportunities for personal and professional development.

### Get in Touch

If you’d like to learn more about our innovative approach to talent acquisition or explore collaboration opportunities, connect with us!

- **Email:** ed [at] edwarddonner [dot] com
- **Website:** [www.edwarddonner.com](http://www.edwarddonner.com)
- **Follow Us:** 
  - [LinkedIn](#)
  - [Twitter](#)
  - [Facebook](#)

### Final Thoughts

Join us on this exciting journey at Edward Donner where technology meets human potential. Together, we can redefine what it means to be fulfilled in the workplace and drive forward the future of recruitment.

In [68]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Brochure

## Welcome to Hugging Face

At Hugging Face, we are a vibrant community dedicated to building the future of AI. Our platform serves as a collaborative hub where developers, researchers, and innovators come together to share models, datasets, and applications. With a diverse range of tools and resources, we empower everyone in the machine learning ecosystem to excel in their projects and endeavors.

---

## Our Platform

- **Models**: Browse over **1M+ models** ranging from text generation to image analysis.
- **Datasets**: Access **250k+ datasets** curated for a variety of machine learning tasks.
- **Spaces**: Create and deploy your applications in real-time; with over **400k apps** currently in use.
- **Enterprise Solutions**: Tailored services for organizations, including enterprise-grade security and dedicated support.

---

## Our Community

Hugging Face is home to over **50,000 organizations**, including key players like:

- Amazon Web Services
- Google
- Microsoft
- Grammarly

These companies leverage our platform to accelerate their AI initiatives and contribute back to the community with cutting-edge models and datasets.

---

## Company Culture

We embrace an open-source ethos, encouraging collaboration and knowledge sharing. At Hugging Face, you'll find:

- **Inclusive Environment**: A culture that values diverse perspectives and backgrounds.
- **Innovation Driven**: Continuous support for research and development.
- **Community Engagement**: Initiatives that promote collaboration among peers in the machine learning space.

---

## Careers

Join our passionate team and help shape the future of AI! We are constantly looking for talented individuals in various roles, including engineering, product management, and research. Check our [careers page](#) for the latest job openings and apply today!

---

### Let's Build the Future of AI Together

Become part of a community where you can learn, share, and innovate. Whether you are a developer, researcher, or an organization, Hugging Face provides you with the tools and support needed to excel in artificial intelligence.

[Explore Hugging Face](https://huggingface.co) | [Sign Up](https://huggingface.co/join)

--- 

*For more information, visit our website or connect with us on social media.* 

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [72]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [70]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Company Brochure

## About Us
Welcome to **Hugging Face** – the AI community building the future. We are a collaborative platform designed to bring together the machine learning community to share and enhance models, datasets, and applications. Our mission is to innovate and accelerate machine learning development, empowering people to create groundbreaking solutions.

![Hugging Face](https://huggingface.co/assets/logo.png)

## Our Offerings
- **Models**: Explore a repository of over **1 million models** ranging from text, image, video, audio, to 3D. 
- **Datasets**: Access and share **250,000+ datasets** tailored to various machine learning tasks.
- **Spaces**: Run and create applications collaboratively, with **400,000+ applications** available.
- **Enterprise Solutions**: Tailored services with enterprise-grade security and dedicated support.

## Community Impact
More than **50,000 organizations**, including industry giants such as Google, Microsoft, AWS, and Meta, use Hugging Face to advance their AI initiatives. We've become a pivotal tool for researchers and developers aiming to push the boundaries of machine learning.

## Company Culture
At Hugging Face, we foster a thriving community culture that emphasizes collaboration, innovation, and inclusivity. Our team is dedicated to building foundational tools for machine learning with the support and contributions of our users. Our environment encourages:
- **Open Source Collaboration**: We believe in the power of community-driven development, where everyone can contribute to and benefit from the latest advancements in ML.
- **Continuous Learning**: Join our diverse team of over 210 professionals committed to personal and collective growth in the evolving field of AI.

## Careers at Hugging Face
Join the **Hugging Face family**! We're always looking for passionate individuals who thrive in creative and collaborative environments. Currently, we have a variety of positions available that span across roles in:
- Engineering
- Research
- Product Management
- Community Engagement

Explore our current job openings and take the next step in your career with us. Visit our [Jobs Page](https://huggingface.co/jobs).

## Join Us
Be part of a rapidly growing community that is shaping the future of AI technology! Whether you are a user, a company seeking enterprise solutions, or a potential recruit, there’s a place for you at Hugging Face.

For more information, visit our website at [Hugging Face](https://huggingface.co) or follow us on social media for updates and community engagement!

--- 

**Contact Us**  
For inquiries, please reach out via our support channels on the website or engage with us on platforms like [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), and [LinkedIn](https://linkedin.com/company/huggingface).

---

Building AI together, one model at a time!

In [71]:
funny_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [75]:
french_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a brochure about the company for prospective customers, investors and recruits. The brochure should be in French language, and Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [76]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": french_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [77]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")


# Hugging Face - La communauté IA qui construit le futur

## Qui sommes-nous?

Hugging Face est une plateforme où la communauté de l'apprentissage automatique collabore sur des modèles, des ensembles de données et des applications. Nous croyons fermement que l'intelligence artificielle a le potentiel de transformer notre avenir et nous nous engageons à construire les outils nécessaires pour y parvenir.

### Nos services

- **Modèles**: Explorez plus de 1 million de modèles pour répondre à vos besoins d'IA.
- **Ensembles de données**: Accédez à plus de 250 000 ensembles de données pour améliorer votre recherche.
- **Espaces**: Collaborer sur des applications IA sans limites avec des ressources optimisées.

## Nos clients

Plus de 50 000 organisations, y compris des géants comme Amazon Web Services, Google, Microsoft, et Grammarly, font confiance à Hugging Face pour leurs solutions d'intelligence artificielle. Nos clients variés nous permettent d'adapter nos outils aux besoins de différents secteurs.

## Culture d'entreprise

Hugging Face valorise un environnement de collaboration où chaque voix compte. Notre culture est centrée sur l'innovation collective, le partage des connaissances et l'utilisation de solutions open source. Nous croyons en la transparence et soutenons un développement inclusif dans le domaine de l'IA.

## Carrières

Nous sommes toujours à la recherche de talents passionnés pour rejoindre notre équipe. Les postes disponibles varient de l'ingénierie logicielle à la recherche en IA. Chez Hugging Face, vous aurez l'opportunité de travailler sur des projets avant-gardistes et de contribuer activement à l'évolution de l'intelligence artificielle.

### Rejoignez-nous !

Pour explorer les possibilités d'emploi et rejoindre notre aventure, consultez notre site [Hugging Face Jobs](https://huggingface.co/jobs).

## Pourquoi choisir Hugging Face?

- **Innovation**: Participez à l'écosystème de l'IA avec des projets à la pointe de la technologie.
- **Communauté**: Collaborez avec d'autres experts dans un environnement inclusif et accueillant.
- **Impact**: Votre travail aura un réel impact sur l'avenir de la technologie IA.

## Contactez-nous

Pour plus d'informations, visitez notre site web ou suivez-nous sur nos réseaux sociaux.

---
**Hugging Face**  
[Site Web](https://huggingface.co) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface)



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>